To solve [day 7](https://adventofcode.com/2020/day/7) of advent of code, you must take a description of nested bags like this one:

```
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
```

And answer a couple of questions about what bags contain the "shiny gold" bag, and what bags are contained by it.

In [43]:
test_input = '''light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
'''

The first thing we can do is recognize the structure of the problem. Each color bag contains 0 or more other colors of bags, so the bag input describes a _graph of bags_

<img src="graph.png" alt="graph" style="width: 600px;"/>

There are lots of ways to represent a graph in a programming language, but we're going to use an **adjacency list**. Our first step in solving the problem is to turn the above description of the problem into a hash table with keys that are bag names and a value that's a hash of bag names and the number of bags contained within.

In [3]:
what_we_want = {
    "light red": {"muted yellow": 2, "bright white": 1},
    "dark orange": {"muted yellow": 4, "bright white": 3},
    "etc": {"etc": 0}
}

There are lots of ways to convert the text into the representation we want. I'll skip complicated regular expressions, and just use split to break up the input:

In [72]:
def parse_bags(input):
    bags = {}
    for line in input.strip().split("\n"):
        # "red bag contains 2 dark olive..." -> ["red bag", "2 dark olive..."]
        color, insides = line.split(" bags contain ")
        bags[color] = {}

        # "2 dark olive bags, 4 yellow bags" -> ["2 dark olive bags", "4 yellow bags"]
        for inside in insides.split(", "):
            # handle the "contain no other bags" case separately
            if 'no other' in inside:
                continue

            # "2 dark olive bags" -> "2 dark olive"
            number_and_color = inside.split(" bag")[0]

            # "2 dark olive" -> ("2", ["dark", "olive"])
            number, *icolor = number_and_color.split(" ")

            # bags["red"]["dark olive"] => 2
            bags[color][' '.join(icolor)] = int(number)

    return bags

bags = parse_bags(test_input)
bags

{'light red': {'bright white': 1, 'muted yellow': 2},
 'dark orange': {'bright white': 3, 'muted yellow': 4},
 'bright white': {'shiny gold': 1},
 'muted yellow': {'shiny gold': 2, 'faded blue': 9},
 'shiny gold': {'dark olive': 1, 'vibrant plum': 2},
 'dark olive': {'faded blue': 3, 'dotted black': 4},
 'vibrant plum': {'faded blue': 5, 'dotted black': 6},
 'faded blue': {},
 'dotted black': {}}

Now that we have our graph represented in a hash, we can look at the actual problems we need to solve.

Part 1 asks us: how many different bag colors could contain the **shiny gold** bag?

In our example, the **bright white** and **muted yellow** bags could contain it directly, and the **dark orange** and **light red** ones could contain them.

The rest of the bags could never contain a shiny gold bag.

To solve this, we can keep start from our shiny gold bag and work outwards, maintaining a list of the already saved bags. If we don't find any new bags, we've found all the bags that can contain our shiny gold bag.

A good first step is to list the bags that can contain a given color. To do so, we'll simply loop through the bags and return the ones that have the given color in them.

In [52]:
# return a list of colors that can contain the given color
def can_contain(color, bags):
    return [c for c in bags if color in bags[c]]

can_contain('shiny gold', bags)

['bright white', 'muted yellow']

Now we can start with shiny gold, search for bags that contain that, and continue that loop until we aren't getting any new bags.

In [73]:
# return a set of bags that can contain a bag, or can 
# contain a bag that contains that bag, and so on recursively
def can_contain_recursive(color, bags):
    valid_bags = {color}
    got_new_bag = True
    
    # iterate until we no longer added any bags to the 
    # valid_bags set
    while got_new_bag:
        got_new_bag = False

        # we're going to be updating the set while we iterate, 
        # so 'list' makes a copy we can iterate on
        for bag in list(valid_bags):
            for container_color in can_contain(bag, bags):
                if container_color not in valid_bags:
                    valid_bags.add(container_color)
                    got_new_bag = True

    # now we can remove the given color, because bags can't 
    # contain themselves in this problem
    return valid_bags - {color}

can_contain_recursive("shiny gold", bags)

{'bright white', 'dark orange', 'light red', 'muted yellow'}

In [105]:
def cc(color, bags):
    return set(c for c in bags if color in bags[c])

def expand(colors, bags):
    return set.union(*[cc(c, bags) for c in colors])

def ccr2(valid_bags, bags):
    all_valid_bags = [expand(bag) for bag in valid_bags]
    return all_valid_bags

def ccr(valid_bags, all_bags):
    print([cc(bag, bags) or set() for bag in valid_bags])
    valid_bags_expanded = set.union(*[cc(bag, bags) for bag in valid_bags] + [set()])
    if valid_bags_expanded != valid_bags:
        valid_bags = ccr(valid_bags_expanded, all_bags)
    return valid_bags

expand({'light red'}, bags)

set()

In [93]:
set.union(*[[] or set()])

set()

Now to solve the problem, we can print the length of the list returned by `can_contain_recursive`

In [56]:
# run can_contain_recursive on the test data
print(len(can_contain_recursive("shiny gold", bags)))

# run can_contain_recursive on my full problem set
print(len(can_contain_recursive("shiny gold", parse_bags(open("input.txt").read()))))

4
355


For part 2 of the problem, we're asked to determine how many bags our bag could contain, and how many bags those bags could contain, and so on.

To do so, we start with one bag, **shiny gold**, and look at the bags it contains: 1 **dark olive** and 2 **vibrant plum**.

For **dark olive**, we add that bag to our sum, then multiply the number of bags contained in _that_ bag by 1.

For **vibrant plum**, we add those 2 bags to our sum, then multiply the number of bags contained in that bag by 2.

When a bag doesn't have any bags inside it, we can simply return zero.

Repeat this procedure, and we'll have a count of all of the many bags that can be contained within one single
bag.

In [71]:
def bag_contains_n_bags(color, bags):
    nbags = 0
    for bag, n in bags[color].items():
        nbags += n + n * bag_contains_n_bags(bag, bags)
    return nbags

# the test set returns 32, as the problem description says
print(bag_contains_n_bags("shiny gold", bags))

# and given my full input, we get 5312 bags contained within the shiny gold bag
print(bag_contains_n_bags("shiny gold", parse_bags(open("input.txt").read())))

32
5312
